<a href="https://colab.research.google.com/github/sankitshane/DeepLearningTF2/blob/master/dog_breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from keras import initializers, layers
from keras.models import Sequential, Model
from keras.layers import (
    Dropout,
    Input,
    AveragePooling2D,
    GlobalAveragePooling2D,
    MaxPool2D
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    ZeroPadding2D
)
from keras.layers.core import (
    Activation,
    Flatten,
    Dense
)
from keras.layers.merge import (
    concatenate,
    add
)
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers.pooling import GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
class AlexNet:
  @staticmethod
  def build(height, width, channel, classes, activation='relu', weightsPath=None):
    model = Sequential()
    inputShape = (height, width, channel)

    if K.image_data_format() == "channel_first":
      inputShape = (channel, height, width)
    
    model.add(Conv2D(96, 11, input_shape=inputShape, strides=(4, 4), padding='valid'))
    model.add(Activation(activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 11, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(384, 3, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(BatchNormalization())

    model.add(Conv2D(384, 3, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 11, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(4096, input_shape=(244*244*3,)))
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

    model.add(Dense(4096))
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

    model.add(Dense(1000))
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

    model.add(Dense(classes))
    model.add(Activation('softmax'))

    if weightsPath is not None:
      model.load_weights(weightsPath)

    return model

In [0]:
def spiltFolder(path, ratio):
  import os
  import shutil
  import random

  mainDir = os.path.join(path, 'Main')
  trainDir = os.path.join(path, 'Train')
  testDir = os.path.join(path, 'Test')
  valDir = os.path.join(path, 'Validation')

  if os.path.isdir(trainDir):
    shutil.rmtree(trainDir)
  if os.path.isdir(testDir):
    shutil.rmtree(testDir)
  if os.path.isdir(valDir):
    shutil.rmtree(valDir)
  
  if not os.path.isdir(mainDir):
    return "Please have the Main Directory Intact"

  shutil.copytree(mainDir, trainDir)
  os.mkdir(testDir)
  os.mkdir(valDir)

  listofcat = os.listdir(trainDir)

  for cat in listofcat:
    os.mkdir(os.path.join(valDir, cat))
    minilist = os.listdir(os.path.join(trainDir, cat))
    minilist_set = int(len(minilist) * (ratio / 100)) * 2

    minilist = random.sample(minilist, minilist_set)
    random_set = random.sample(minilist, minilist_set//2)

    test_minilist = {}

    for img in random_set:
      test_minilist[img] = True
      shutil.move(os.path.join(trainDir, cat, img), os.path.join(testDir, img))
    for img in minilist:
      if img not in test_minilist:
        shutil.move(os.path.join(trainDir, cat, img), os.path.join(valDir, cat, img))
    
  return "Image Rearranged"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/DLData/Train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 9084 images belonging to 120 classes.


In [0]:
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/DLData/validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800)